# Alternativ lab 2 DP-300 kursus

Formål:Deploy Azure SQL Database, Firewall rules, Allow Azure Services, Service objectives, Elastic pool, Change Service tier

## Oprettelse af en ny Azure SQL Server og database via portalen

1. Åbn Azure portalen ([https://portal.azure.com](https://portal.azure.com/)) i din browser
    
2. Opret en ny ressource gruppe ved at klikke på Create a resource - søg efter resource group
    

> > Resource Group Name: DK
> 
> > Region: West Europe

3. Opret en ny resource: SQL Server (logical server)

> > Resource Group: DK
> 
> > Server name:sql-\<dine initialer\>-\<yyyymmdd\> f.eks. sql-cg-20211108
> 
> > Location:West Europe
> 
> > Behold default indstillingen forAuthentication method (Use SQL authentication)
> 
> > Server admin login:sqldbaadmin
> 
> > Password:Super4U! (NB læg mærke til 8 tegn er nok her...)
> 
> > Under Networking Behold default Firewall rules: Allow Azure Services: No
> 
> > MicrosoftDefender for SQL: Not now
> 
> > Bemærk serveren er bare en logisk server på en given lokation - vi har ikke taget stilling til service og compute tier dvs virtuel CPU/RAM, data størrelse og afregningsmodel og dermed pris. Det foregår på databaseniveau.

4. Opret en ny resource: SQL Database

> > Resource group: DK
> 
> > Database name: AdventureworksCloudDB
> 
> > Server:Den du lige har oprettet i punkt 3 (NB man kan også lave en ny server adhoc her)
> 
> > Elastic pool: No
> 
> > Compute+storage: Klik på Configure Database
> > 
> > Undersøg de forskellige Service tier muligheder
> > 
> > Bemærk: Read scale-out mulighed er Enabled på Business Critical og på den gamle Premium
> > 
> > Bemærk: Hyperscale advarsel - kan ikke ændre derfra og til de andre service tier muligheder
> > 
> > Bemærk: Mulighed for at vælge Hardware configuration med VM
> > 
> > Se yderligere info på [https://docs.microsoft.com/en-us/azure/azure-sql/database/resource-limits-vcore-single-databases](https://docs.microsoft.com/en-us/azure/azure-sql/database/resource-limits-vcore-single-databases)
> > 
> > Hvis man scripter oprettelsen af en database via SQL/PowerShell angiver man service tier

> <span style="font-size: 14px;">Vælg Service tier:General Purpose</span>

> > <span style="font-size: 14px;">Compute tier: Serverless (billigste løsning for os, når det ikke er et produktionsmiljø med mange forespørgsler)<br></span>Min vCores: 5  
> > Max vCores: 40  
> > Zone redundant: No
> 
> <span style="font-size: 14px;">Backup storage redundancy: Geo-redundant backup storage (Dermed gemmes vores automatiske database backup også i North Europe (Ireland) som er makker regionen til West Europe (Holland.</span>
> 
> Networking: Bemærk muligheden for at tilføje din nuværende ipadresse her - vi vil hellere håndtere det på databaseniveau
> 
> Networking: Bemærk muligheden for at registrere et private endpoint - det håndteres på sql server niveau og afgrænser adgangen til ipadresser på et virtuelt netværk i Azure. Vi lægger denne restriktion på senere i dette Lab.
> 
> Security: Defender og Ledger: Not now
> 
> Additonal settings: Data source -\> Use existing data: Sample  
> Database collation:Læg mærke til den låser sig fast på SQL\_Latin1\_General\_CP1\_CI\_AS, når vi vælger at tage udgangspunkt i en eksisterende database. For nye database kan man vælge f.eks. Danish\_Norwegian\_CI\_AS

<span style="font-size: 14px;">5) Find sql serveren i listen af ressourcer i Azure og klik ind på den. Under overview kan du se navnet på Server admin og Server name</span>

<span style="font-size: 14px;">6) Forsøg at forbinde til Azure SQL serveren fra SQL Server Management Studio (SSMS) på din lokale maskine. Anvend SQL Server Authentication.</span>

> <span style="font-size: 14px;">Server Admin: sqldbaadmin</span>
> 
> <span style="font-size: 14px;">Password: Super4U!</span>

<span style="font-size: 14px;">7) Forbindelsen mislykkes, men SSMS påpeger at det nok er en firewall regel som mangler. Vælg cancel i SSMS.</span>

<span style="font-size: 14px;">8) Klik på Show firewall settings i Azure portal for din sql server under Overview</span>

<span style="font-size: 14px;">9) Klik på knappen Add client IP i toppen af billedet. Navngiv reglen "SuperUsers Hillerød" i stedet for det automatiske navn som foreslåes. Klik Save for at gemme reglen.</span>

<span style="font-size: 14px;">10) Vend tilbage til SSMS og test at du nu fint kan forbinde til din Azure SQL server. Den dukker op som en blå spand ude til venstre i Object Explorer.</span>

<span style="font-size: 14px;">11) Klik på den blå spand og vælg New Query i toolbaren eller via højreklik. Du står nu på master databasen.</span>

<span style="font-size: 14px;">12) Tjek hvilke databaser du har ved at køre følgende statement:&nbsp;</span>

In [ ]:
SELECT      *
FROM        sys.databases

Vi har kun <span style="color: #09885a;">2</span> databaser i form af <span style="color: #0000ff;">master</span> databasen og AdventureworksCloudDB. Tempdb findes også, men vi administrerer den ikke. Her er lidt info om tempdb og størrelsen på den. Se https:<span style="color: #000000;">//</span>docs.microsoft.com<span style="color: #000000;">/</span>en<span style="color: #000000;">-</span>us<span style="color: #000000;">/</span><span style="color: #0000ff;">sql</span><span style="color: #000000;">/</span>relational<span style="color: #000000;">-</span>databases<span style="color: #000000;">/</span>databases<span style="color: #000000;">/</span>tempdb<span style="color: #000000;">-</span><span style="color: #0000ff;">database</span>?view<span style="color: #000000;">=</span><span style="color: #0000ff;">sql</span><span style="color: #000000;">-</span><span style="color: #0000ff;">server</span><span style="color: #000000;">-</span>ver15#tempdb<span style="color: #000000;">-</span><span style="color: #0000ff;">database</span><span style="color: #000000;">-</span><span style="color: #0000ff;">in</span><span style="color: #000000;">-</span><span style="color: #0000ff;">sql</span><span style="color: #000000;">-</span><span style="color: #0000ff;">database</span>

<span style="color: #0000ff;">13) Find hvilke firewall regler der er defineret på server niveau via denne query. Der skulle gerne være en enkelt regel som du oprettede via portalen:</span>

In [ ]:
SELECT		*
FROM		sys.firewall_rules

14. Gå tilbage til portalen og sæt Yes ud for "Allow Azure services and resources to access this server" under Firewalls and virtual networks for din sql server. Husk at klikke Save for at gemme indstillingen.
15. Kør query'en som viser server firewall regler igen - og bid mærke i den nye 4x0 regel **AllowAllWindowsAzureIps**. Den tillader adgang fra **ALLE** maskiner som har en ipadresse i et Azure Datacenter hvor som helst på jorden!

In [ ]:
SELECT		*
FROM		sys.firewall_rules

16. Forbind med remote desktop til den Azure VM vi oprettede i Lab 1.   
    Login: dbadmin  
    Password:SuperUsers1234!!
    
17. Åbn SSMS og forsøg at forbinde til din egen eller til en medkursists Azure SQL Server. Du vil kunne forbinde til din nabos, hvis han/hun har oprettet 4x0 reglen herover!
    
    Server navn: Din eller gerne din medkursists servernavn  
    Server Admin: sqldbaadmin  
    Password: Super4U!
    
18. Opret en ny Azure SQL Database via SSMS på den server du er logget ind på. Hvis du højreklikker på Databases i Object Explorer kan du vælge om du vil scripte direkte som SQL eller indtaste i et vindue hhv New Database script / New database... (TIP Læs eventuelt allerede punkt 20 nu som giver hjælp til scripting)
    
    > Collation:Danish\_norwegian\_CI\_AS  
    > Name: TheHandel  
    > Edition:GeneralPurpose  
    > Service Objective:GP\_Gen5\_2  
    > Max Size: 20 GB
    
19. Opret endnu en database via scriptet her. Bemærk S'et i GP\_**S**\_Gen5\_2 - det betyder Serverless:

In [ ]:
CREATE DATABASE Kaffehandel
COLLATE DANISH_NORWEGIAN_CI_AS
(
	 EDITION = 'GeneralPurpose'
	,SERVICE_OBJECTIVE = 'GP_S_Gen5_2'
	, MAXSIZE = 20 GB
);

20. Hvis du markerer CREATE DATABASE i SSMS og taster F1 åbner din browser en hjælpeartikel med Microsofts officielle dokumentation til SQL Server. Bemærk at der er en CREATE DATABASE artikel til de forskellige produkter: SQL Server (on premise), SQL Database, Managed Instance, Synapse etc. Vælg SQL Database som produkt.
21. Bid mærke i de mange forskellige Service objectives ELLER man kan angive en ELASTIC\_POOL. Der er også en ekstra funktionalitet for Azure SQL databaser, som gør det meget nemt at kopiere en database via CREATE DATABASE-COPY syntaksen i boks 2.
22. Opret en database mere via COPY syntaksen:

In [ ]:
CREATE DATABASE AdvCloudKopi
AS COPY OF AdventureworksCloudDB
(
	SERVICE_OBJECTIVE = 'GP_S_Gen5_1'
)


23. Man kan finde Service objetive for en database via SQL funktionen DATABASEPROPERTYEX og øvrige databaseindstillinger således. NB Man skal stå på den aktuelle database - står man på master returneres der bare NULL:

In [ ]:
SELECT		DATABASEPROPERTYEX('AdventureWorksCloudDB','Edition')
			,DATABASEPROPERTYEX('AdventureworksCloudDB','MaxSizeInBytes')
			,DATABASEPROPERTYEX('AdventureworksCloudDB','ServiceObjective')


24) Sæt DBA group som Azure AD admin for din sql server i portalen